# Jupyter Notebook Image Classifier - Transfer Learning
**Model Class:**    Supervised</br>
**Model Type:**     Classification Using Transfer Learning</br>

**Purpose** : This notebook provides a template for developing a deep learning image classification model using the transfer learning technique. Transfer learning is a deep learning technique where the optimized weights from one neural network are reused as the starting weights for another neural network. In this example, we focus on developing an image classification model on our local machine using the weights from the VGG-19 pre-trained Model. This would usually take a long time to train from scratch, but using pre trained weights significantly reduces the time and compute power required to train a new model. The three classes that we are training for in this notebook are: 
    * Bird
    * Person
    * Plant

With the exception of data preparation and training data set up, this notebook can be used as is with little change to the training and evaluation code to build an image classification model. 

**Edit Date**: 10/22/2018

**NOTE**: The methods described in this notebook are suitable for smaller datasets similar to the pretrained dataset. For larger datasets please refer to the *Jupyter Notebook Image Classifier* range of notebooks.

This notebook was trained on an NVIDIA TESLA K80 GPU with tensorflow-gpu, CUDA 9.2 and cuDNN 7.2.1 support. If this notebook is run on the cpu of a personal computer, the expected runtime will be significatly higher and the load on the system will also be high.
</br></br>
- ##### **Classification models included in this notebook:**
    - Convolutional Neural Network (CNN)
</br></br>
- **Resources**
    * Hands-On Machine Learning with Scikit-Learn & TensorFlow - Aurélien Géron
    * Deep Learning - Josh Patterson & Adam Gibson
    * Deep Learning with Tensorflow and Keras - Antonio Gulli & Sujit Pal
</br></br>
- **Data**</br>
    - The image data can be downloaded from the following URL: http://image-net.org/download-images
    - If you are unable to download the data, reach out to the Artificial Intelligence Accelerator through the spark page. 
    - The following classes are present in the dataset:
        * Bird    *(n01503061)*
        * Person  *(n00007846)*
        * Plant   *(n00017222)*

- **Goal**</br>
    - Predict the class of an image that contains one of the objects outlined above.
    
# Dependencies

Load the dependencies for model development. Current package requirements include:
* Keras
* Tensorflow
* Sklearn
* CV
* Pandas
* Numpy
* Matplotlib

The current template uses the following versions:
* Keras version: 2.2.4
* Tensorflow version: 1.8.0
* Sklearn version: 0.19.2
* CV version: 3.4.3
* Pandas version: 0.23.4
* Numpy version: 1.15.2
* Matplotlib version: 2.1.2

In [ ]:
import keras
import sklearn
import matplotlib
import numpy as np
import pandas as pd
import cv2
import os

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix,accuracy_score,auc,roc_auc_score,roc_curve
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPool2D, Activation, Flatten, MaxPooling2D, Convolution2D
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.optimizers import SGD, adam, adagrad, RMSprop
from keras.preprocessing.image import img_to_array
from keras.models import Model
from keras.applications.vgg19 import VGG19,preprocess_input
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

from random import shuffle

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
print ("Keras version: " + keras.__version__)
print ("Tensorflow version: " + tf.__version__)
print ("Sklearn version: "+sklearn.__version__)
print ("CV version: "+cv2.__version__)
print ("Pandas version: "+pd.__version__)
print ("Numpy version: "+np.__version__)
print ("Matplotlib version: "+matplotlib.__version__)

# Load Data

### Read the data
The first step is to import the dataset. Since we are dealing with unstructured data, specifically images, there are several ways to do this. The current notebook uses a dataset which can be loaded from your local machine.

In [ ]:
#here the data is stored in three separate folders each with only one image class
#this method will list out all the elements in the directory which makes them easy to import
bird_path = './Data/n01503061'
person_path= './Data/n00007846'
plant_path= './Data/n00017222'

bird_dir = os.listdir(bird_path)
person_dir = os.listdir(person_path)
plant_dir = os.listdir(plant_path)

#Defining the paths for the image
for i in range(len(bird_dir)):
    bird_dir[i] = './Data/n01503061/'+bird_dir[i]
for i in range(len(person_dir)):
    person_dir[i] = './Data/n00007846/'+person_dir[i]
for i in range(len(plant_dir)):
    plant_dir[i] = './Data/n00017222/'+plant_dir[i]

# Data Preparation & Transformation

Read in a data set and clean/prepare it for modelling. This activity will heavily depend on your specific problem and the data set that you are using. When possible, a best practice should be to include transformation, scaling, and normalization activities as a function withing your model pipeline. This will simplify the deployment step but is not absolutely necessary.


In [ ]:
# Creating class labels for the data
bird_name = []
person_name = []
plant_name = []
for i in range(len(bird_dir)):
    bird_name.append('Bird')
for i in range(len(person_dir)):
    person_name.append('Person')
for i in range(len(plant_dir)):
    plant_name.append('Plant')

In [ ]:
# This will group the data with their label in a zip data structure
birds = list(zip(bird_dir, bird_name))
people = list(zip(person_dir,person_name))
plants = list(zip(plant_dir,plant_name))

data = birds+ people+ plants

In [ ]:
# This randomly shuffles the data for better sampling 
shuffle(data)
addrs, labels = zip(*data)

In [ ]:
labels = np.array(labels)

### Loading the data and preprocessing

In [ ]:
dataset = []
i = 0
# This method will help you load data as numpy arrays
for image_paths in addrs:
    
    image = cv2.imread(image_paths)
    image = cv2.resize(image, (224,224))
    image = img_to_array(image)
    image = preprocess_input(image)
    dataset.append(image)
    

In [ ]:
# Format the data
dataset = np.array(dataset, dtype = 'int32')
dataset.shape

In [ ]:
# Lets take alook at the images:
image = cv2.imread(data[3900][0])
# OpenCV reads images as BGR channels. In order to view images properly, we canuse cvtColor function to revert to regular RGB channels
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
plt.imshow(image)
print(labels[3900])

In [ ]:
# Label encoding
lb = LabelEncoder()
labels = lb.fit_transform(labels)

In [ ]:
np.unique(labels)
nb_classes = 3

# Create Training and Validation Data Sets
Create the training and test splits that will be used to train and evaluate the model. Some considerations that may impact model performance when creating the train test splits include:
* Class balance - Are the classes well balanced or should the data set be resampled prior to model development
* Split Size - How large are the training and test splits. This will depend on the sample size and training strategy including using a cross validation
* Random Seed - For reproducable experiments, a consistent random seed should be used

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(dataset, labels, random_state = 42, test_size = 0.2)

In [ ]:
#normalize the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

Y_train = np_utils.to_categorical(y_train,3)
Y_test = np_utils.to_categorical(y_test,3)

In [ ]:
print("Training Data shape: ",x_train.shape)
print("Testing Data shape: ",x_test.shape)
print("Training labels shape: ",y_train.shape)
print("Testing labels shape: ",y_test.shape)

print("Training labels shape: ",Y_train.shape)
print("Testing labels shape: ",Y_test.shape)

# Build the Models

Here we have built two models. The first is a custom CNN model and the second is the VGG-19 model. The VGG-19 model only had its convoloutional layers imported and a fully connected layer is built on top of that.

In [ ]:
def convNet_classifier(optimizer = 'adam'):
    # This is a custom cnn model used to illustrate the difference between custom models and 
    # models trained with transfer learning
    n_hidden = 128
    nb_classes = 3
    # create convnet
    model = Sequential()
    model.add(Conv2D(32, kernel_size= (3,3), padding= "same", input_shape = (224, 224, 3), activation= "relu"))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, kernel_size= (3,3), padding= "same", activation= "relu"))
    model.add(MaxPooling2D((2,2)))
    model.add(Conv2D(64, kernel_size= (3,3), padding= "same", activation= "relu"))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(32, kernel_size= (3,3), padding= "same", input_shape = (224, 224, 3), activation= "relu"))
    model.add(MaxPooling2D((2,2)))
    
    # Create the dense layer
    model.add(Flatten())
    model.add(Dense(512, kernel_initializer= 'normal'))
    model.add(Activation("relu"))
    model.add(Dropout(0.2))
    model.add(Dense(256, kernel_initializer= 'normal'))
    model.add(Activation("relu"))
    model.add(Dropout(0.2))
    model.add(Dense(nb_classes))
    model.add(Activation("softmax"))
    
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = 'adagrad', metrics = ["accuracy"])
    return model

checkpoint = ModelCheckpoint("vgg19_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [ ]:
# Model for Transfer Learning
model = VGG19(weights = 'imagenet', include_top = False, input_shape=(224,224,3))
model.summary()

In [ ]:
for layers in model.layers:
    layers.trainable = False

In [ ]:
# since the data we have has fewer images training an entire model will result in overfitting. 
# to avoid this we simple train the top layer while freezing the convolutional layers
# here we will define our custom FC Layer

x = model.output
x = Flatten()(x)
x = Dense(1024, activation='relu', kernel_initializer= 'normal')(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu', kernel_initializer= 'normal')(x)
x = Dropout(0.2)(x)
x = Dense(units = 3, activation= 'softmax')(x)

final_model = Model(input= model.input, output= x)

final_model.compile(loss = 'sparse_categorical_crossentropy', optimizer = SGD(lr = 0.0001, momentum= 0.9), metrics = ['accuracy'])
final_model.summary()

# Build Optimization Pipeline

### Optimizing hyperparameters with gridsearch

Set up the model pipeline. Each pipeline should take a model at a minimum. A best practice would be to include the data preparation activities as a step in the pipeline to allow for easier saving and production implementation of the model after training. The pipeline setup will include specifying the classifer, setting up the parameters to explore for grid search, and creating the grid search objects. A reference to pipeline variations is available at the following link:

* http://queirozf.com/entries/scikit-learn-pipeline-examples#pipeline-with-cross-validation-gridsearchcv

In [ ]:
pipe_cnn = KerasClassifier(build_fn= convNet_classifier, epochs = 20, batch_size = 30, verbose=1)

##### Create a grid for each model and add the hyperparameters that you want to vary during the optimization process

In [ ]:
#Defining the Grid Parameters
grid_params_cnn = [{}]

#### Setup the grid search

Set up the grid search object for each model. The grid search object takes a pipeline, scoring critera, number of jobs (for parallelizing the search), and data folds (for cross validation on training set). The resulting grid objects should be added to a list for training purposes and a dictionary with IDs for easy recall and presentation after evaluation

In [ ]:
# model training configuration
folds = 3 # number of data folds for validation
jobs = -1 # number of jobs for 
scoring = 'accuracy'

In [ ]:
gs_cnn = GridSearchCV(estimator= pipe_cnn,
                     param_grid = grid_params_cnn,
                     scoring= scoring,
                     cv= folds, n_jobs= jobs)

In [ ]:
grid_dict_cnn = {0: "LeNet"}
grid_CNN = [gs_cnn]

# Model Training
#### Training a model with GridSearch
Perform a hyperparameter grid search and select the model that performs the best according to specified criteria.

In [ ]:
def greater(x,y):
    if x>y:
        return True
    else:
        return False
def less(x,y):
    if x<y:
        return True
    else:
        return False
    

def trainModel(grids,grid_dicts,X_train,X_test,y_train,y_test,eval_method = 'accuracy'):
    
    modelEval={'accuracy': [metrics.accuracy_score,greater],
               'f1': [metrics.f1_score,greater],
               'recall': [metrics.recall_score,greater],
               'precision': [metrics.precision_score,greater],
               'cross_entropy': [metrics.log_loss,less], 
    }
    
    best_score = 0.0
    best_clf = None
    best_gs = ''
    best_per_grid = []
    
    print('Running model optimizations...')
    for idx, gs in enumerate(grids):
        print('\nEstimator: %s' % grid_dicts[idx])
        try:
            # Fit grid search
            gs.fit(X_train, y_train)
        except:
            print("\n******************\nIssue running the %s model, review your code and re run \n******************" %grid_dict[idx]) 
            continue
        
        # Best params
        print('Best params: %s' % gs.best_params_)
        # Best training data accuracy
        
        print('Best training accuracy: %.3f' % gs.best_score_)
        # Predict on test data with best params
        y_pred = gs.predict(X_test)
        
        # run the evaluation
        score = modelEval[eval_method][0](y_test,y_pred)
        
        # Test data accuracy of model with best params
        print("Test set "+ eval_method+" score for best params: %.3f " % score)
        
        # track the best model
        if modelEval[eval_method][1](score,best_score):
            best_score = score
            best_gs = gs
            best_clf = idx
            
        # store the best parameter set for this model 
        best_per_grid.append([idx,gs.best_score_,gs.best_params_])
    
    return best_score,best_gs,best_clf,best_per_grid

**Training the Custom Model**

In [ ]:
# Train the model
best_score,best_grid,best_model,best_per_model = trainModel(grid_CNN,grid_CNN,x_train,x_test,y_train,y_test)

if best_model != None:
    print('\nClassifier with best test set accuracy: %s with a score of %.3f' % (grid_dict[best_model], best_score))    

else:
    print('\n--------------------------------\nIssue running models, review code and re run')

**Training the VGG-19 Model for transfer learning**

In [ ]:
final_model.fit(x_train, y_train, batch_size= 28, epochs= 10, validation_split= 0.1, verbose= 1)

# Save the models

Save the models so that they can be read by other applications or deployed into production. Model weights, objects, and/or pipelines are dumped to disk as json and h5 files for easy access and reusability

### Save the custom CNN model

In [ ]:
conv_model_json = best_grid.to_json()
open('Image_Classification_CNN_Architecture','w').write(conv_model_json)
best_grid.save_weights('Image_Classification_CNN_Weights.h5')

### Save model that was built using the VGG-19 pre-trained weights

The VGG-19 model only had its convoloutional layers imported and a fully connected layer was built on top of that.

In [ ]:
conv_model_pre_trained_json = final_model.to_json()
open('Image_Classification_CNN_PreTrained_Architecture','w').write(conv_model_pre_trained_json)
final_model.save_weights('Image_Classification_CNN_PreTrained_Weights.h5')

<hr/>
# Appendix

## Generic process to read image data from local repository

### Load Data

The first step is to import the dataset. Since we are dealing with unstructured data, specifically images, there are several ways to do this. The current notebook uses a dataset which can be loaded from your local machine.

In [ ]:
# Here the data is stored in three separate folders each with only one image class
# this method will list out all the elements in the directory which makes them easy to import
bird_path = './Data/n01503061'
person_path= './Data/n00007846'
plant_path= './Data/n00017222'

bird_dir = os.listdir(bird_path)
person_dir = os.listdir(person_path)
plant_dir = os.listdir(plant_path)

# Defining the paths for the image
for i in range(len(bird_dir)):
    bird_dir[i] = './Data/n01503061/'+bird_dir[i]
for i in range(len(person_dir)):
    person_dir[i] = './Data/n00007846/'+person_dir[i]
for i in range(len(plant_dir)):
    plant_dir[i] = './Data/n00017222/'+plant_dir[i]

# Data Preparation & Transformation

Read in a data set and clean/prepare it for modelling. This activity will heavily depend on your specific problem and the data set that you are using. When possible, a best practice should be to include transformation, scaling, and normalization activities as a function withing your model pipeline. This will simplify the deployment step but is not absolutely necessary.


In [ ]:
# Creating class labels for the data
bird_name = []
person_name = []
plant_name = []
for i in range(len(bird_dir)):
    bird_name.append('Bird')
for i in range(len(person_dir)):
    person_name.append('Person')
for i in range(len(plant_dir)):
    plant_name.append('Plant')

In [ ]:
# This will group the data with their label in a zip data structure
birds = list(zip(bird_dir, bird_name))
people = list(zip(person_dir,person_name))
plants = list(zip(plant_dir,plant_name))

data = birds+ people+ plants

In [ ]:
# This randomly shuffles the data for better sampling 
shuffle(data)
addrs, labels = zip(*data)

In [ ]:
labels = np.array(labels)

### Loading the data and preprocessing

In [ ]:
dataset = []
i = 0
# This method will help you load data as numpy arrays
for image_paths in addrs:  
    image = cv2.imread(image_paths)
    image = cv2.resize(image, (224,224))
    image = img_to_array(image)
    image = preprocess_input(image)
    dataset.append(image)
    

In [ ]:
# Format the data
dataset = np.array(dataset, dtype = 'int32')
dataset.shape

In [ ]:
# Lets take alook at the images:
image = cv2.imread(data[3900][0])
# OpenCV reads images as BGR channels. In order to view images properly, we canuse cvtColor function to revert to regular RGB channels
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
plt.imshow(image)
print(labels[3900])

In [ ]:
# Label encoding
lb = LabelEncoder()
labels = lb.fit_transform(labels)

In [ ]:
np.unique(labels)
nb_classes = 3

# Create Training and Validation Data Sets
Create the training and test splits that will be used to train and evaluate the model. Some considerations that may impact model performance when creating the train test splits include:
* Class balance - Are the classes well balanced or should the data set be resampled prior to model development
* Split Size - How large are the training and test splits. This will depend on the sample size and training strategy including using a cross validation
* Random Seed - For reproducable experiments, a consistent random seed should be used

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(dataset, labels, random_state = 42, test_size = 0.2)

In [ ]:
#normalize the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

Y_train = np_utils.to_categorical(y_train,3)
Y_test = np_utils.to_categorical(y_test,3)

In [ ]:
print("Training Data shape: ",x_train.shape)
print("Testing Data shape: ",x_test.shape)